In [1]:
#Обновим и подключим библиотеки:
!pip install --upgrade pip
!pip install --upgrade scikit-learn==0.23.0

     |████████████████████████████████| 1.5MB 2.8MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 7.3 MB 2.8 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [2]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
#Импортируем датасет MNIST.
from keras.datasets import mnist
(X_train, y_train), (X_pred, y_pred) = mnist.load_data()

Using TensorFlow backend.


11493376/11490434 [==============================] - 0s 0us/step


In [4]:
#Объем тренировочного набора данных:
len(X_train)

60000

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
#Для начала преобразуем данные в привычный формат.
dim = 784 # 28*28 
X_train = X_train.reshape(len(X_train), dim)
X_pred = X_pred.reshape(len(X_pred), dim)

In [ ]:
pca = PCA(n_components=784, svd_solver='full')
X_full = pca.fit(X_train).transform(X_train)

1. Определите, какое минимальное количество главных компонент M необходимо использовать, чтобы доля объясненной дисперсии превышала 0.71 (была строго больше указанного значения).

In [ ]:
explained_variance = np.round(np.cumsum(pca.explained_variance_ratio_),3)

checker = 0
for i in range(784):
    if explained_variance[i] > 0.71:
        checker = i
        break
print(checker+1)

28


2. Найдите счеты, отвечающие найденным в предыдущем пункте 
M
 главным компонентам: PCA(n_components=M, svd_solver='full').

In [ ]:
#Счеты находятся просто библиотекой, как это было в первой недели курса. Итак, определив необходимое число ГК вы сокращаете размерность тренировочного набора данных, число ГК вы также определяете на нем. Пусть ГК будет 28, тогда код:
pca = PCA(n_components=28, svd_solver='full')
modelPCA = pca.fit(X_train) 

3. Разделите полученную выборку (после сокращения размерности) случайным образом в отношении 70/30: train_test_split(X_train, y_train, test_size=0.3, random_state=54).

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.3, random_state=54)

In [ ]:
#позволяет нам получить преобразование, а уже 
X_train = modelPCA.transform(X_train)
#снижает размерность данных X_train. Когда необходимо снизить размерность тестовых данных или просто новых данных, мы используем имеющееся преобразование modelPCA к ним:
X_test = modelPCA.transform(X_test)

Введите выборочное среднее нулевой колонки для полученного тренировочного набора:

In [ ]:
X_train[:,0].mean() #строка это X_train[0], стобец это X_train[:,0] - нужно посчитать столбец. Ответ 0.844481936070177

0.844481936070177

4. Используя метод многоклассовой классификации One-vs-All OneVsRestClassifier(), обучите алгоритм случайного леса RandomForestClassifier() с параметрами criterion='gini', min_samples_leaf=10, max_depth=20, n_estimators=10, random_state=54. Выполните оценку с помощью тестовых данных.

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
#Обучаем модель:
ensemble = RandomForestClassifier(criterion='gini', min_samples_leaf=10, max_depth=20, n_estimators=10, random_state=54)
clf_forest = OneVsRestClassifier(ensemble).fit(X_train, y_train)

In [ ]:
y_pred = clf_forest.predict(X_test)

In [ ]:
#Выполняем оценку:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[1742,    0,    6,    4,    4,   10,   21,    3,    4,    3],
       [   1, 1992,    9,    8,    1,   13,    1,    3,    4,    1],
       [  17,    7, 1650,   14,   21,    8,    7,   15,   28,    8],
       [   6,    5,   40, 1627,    4,   33,   15,   21,   50,   12],
       [   9,    9,   12,    1, 1591,    9,   21,    6,   12,   82],
       [  13,    3,    9,   60,   23, 1439,   28,   10,   28,   15],
       [  11,    7,   16,    1,    4,   27, 1724,    0,    8,    1],
       [   8,   15,   27,    1,   15,    3,    1, 1762,    6,   45],
       [  12,   17,   19,   69,    8,   53,   19,   14, 1502,   32],
       [  12,    7,   10,   32,   80,   15,    5,   45,   13, 1556]])

Введите количество верно классифицированных объектов класса 4 для тестовых данных:

5. Используя метод многоклассовой классификации One-vs-All OneVsRestClassifier(), обучите алгоритм логистической регрессии LogisticRegression(solver='lbfgs', random_state=54). Остальные параметры оставьте по умолчанию. Выполните оценку с помощью тестовых данных.

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
#Обучаем модель:
linear_model = LogisticRegression(solver='lbfgs', random_state=54)
clf_logis = OneVsRestClassifier(linear_model).fit(X_train, y_train)

In [ ]:
y_pred = clf_logis.predict(X_test)

In [ ]:
#Выполняем оценку:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[1727,    0,   11,    4,    3,   17,   12,    4,   15,    4],
       [   1, 1974,   10,    6,    0,   17,    3,    2,   18,    2],
       [  13,   23, 1507,   30,   46,   10,   36,   28,   47,   35],
       [  14,   15,   56, 1515,    5,   90,   19,   29,   44,   26],
       [   7,   16,   11,    5, 1564,    8,   21,    6,   26,   88],
       [  43,   10,   19,  107,   42, 1244,   50,   10,   62,   41],
       [  12,    6,   17,    3,   18,   38, 1684,    1,   17,    3],
       [  10,   10,   39,    5,   30,    6,    2, 1723,    4,   54],
       [  16,   42,   44,   77,   13,   79,   14,   10, 1397,   53],
       [  16,   12,   20,   39,  101,   31,    1,   97,   24, 1434]])

Введите количество верно классифицированных объектов класса 6 для тестовых данных:

6. Используя метод многоклассовой классификации One-vs-All OneVsRestClassifier(), обучите дерево принятия решений DecisionTreeClassifier() с параметрами criterion='gini', min_samples_leaf=10, max_depth=20, random_state=54. Выполните оценку с помощью тестовых данных.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
#Обучаем модель:
tree = DecisionTreeClassifier(criterion='gini', min_samples_leaf=10, max_depth=20, random_state=54)
clf_tree = OneVsRestClassifier(tree).fit(X_train, y_train)

In [ ]:
y_pred = clf_tree.predict(X_test)

In [ ]:
#Выполняем оценку:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[1624,    0,   32,   21,    8,   36,   30,   10,   16,   20],
       [   0, 1967,   22,    8,    1,   12,    4,    3,    6,   10],
       [  27,   19, 1473,   37,   19,   25,   35,   30,   51,   59],
       [  20,   13,   77, 1477,    7,   70,   16,   25,   60,   48],
       [  14,   10,   36,    4, 1464,   16,   33,   21,   28,  126],
       [  24,    4,   75,   88,   31, 1253,   34,   15,   57,   47],
       [  23,    7,   29,    8,   12,   29, 1638,    3,   20,   30],
       [   6,   26,   35,   11,   21,   10,    1, 1661,   19,   93],
       [  18,   23,   92,   83,   12,   70,   14,   15, 1358,   60],
       [  17,    9,   51,   35,  119,   34,    3,   70,   36, 1401]])

Введите количество верно классифицированных объектов класса 3 для тестовых данных:

7. Примените полученное ранее преобразование метода главных компонент к новым данным (идентификаторы строк соответствуют файлам изображений).

In [ ]:
data = 'https://courses.openedu.ru/assets/courseware/v1/25c0abab1076236b7427fac937c5663e/asset-v1:ITMOUniversity+ADVML+spring_2020_ITMO_mag+type@asset+block/pred_for_task.csv'

In [ ]:
import pandas as pd
# load dataset into Pandas DataFrame
data = pd.read_csv(data)

In [ ]:
data_test = data.drop('FileName', axis=1)

In [ ]:
data

,FileName,Label,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,...,f744,f745,f746,f747,f748,f749,f750,f751,f752,f753,f754,f755,f756,f757,f758,f759,f760,f761,f762,f763,f764,f765,f766,f767,f768,f769,f770,f771,f772,f773,f774,f775,f776,f777,f778,f779,f780,f781,f782,f783
0,file1,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,file2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,file3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,file4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,file5,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,file6,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,file7,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,file8,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,file9,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,file10,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


8. Выполните предсказание для указанных изображений цифр с помощью обученных алгоритмов.


Введите вероятность отнесения изображения file20 к назначенному классу алгоритмом случайного леса.

In [ ]:
#Предсказание для file20 из таблицы
clf_forest.fit(X_train, y_train)
#Bыбор строки из таблицы. Строка начинается с нулевого индекса
File20 = data_test.loc[19,:].to_frame().T
File20 = File20.drop(['Label'], axis=1)
File20 = modelPCA.transform(File20)
#Oтбор данных для предикторов и предсказание
clf_forest.predict_proba(File20)

array([[0.        , 0.        , 0.01365112, 0.        , 0.93830588,
        0.        , 0.        , 0.        , 0.        , 0.048043  ]])

In [ ]:
#Округляем ответ до 1000, при этом File20 содержит только одну нулевую строку, где первый [0] означает нулевая строка, а второй [0] номер столбца вектора 0 до 9.
#Необходимо найти вероятность отнесения изображения 4 к ее реальному значению (4), это четвертый столбец.
print(round(clf_forest.predict_proba(File20)[0][4],3))

0.938


Введите вероятность отнесения изображения file24 к назначенному классу алгоритмом логистической регрессией.

In [ ]:
#Предсказание для file24 из таблицы
clf_logis.fit(X_train, y_train)
#Bыбор строки из таблицы
File24 = data_test.loc[23,:].to_frame().T
File24 = File24.drop(['Label'], axis=1)
File24 = modelPCA.transform(File24)
#Oтбор данных для предикторов и предсказание
clf_logis.predict_proba(File24)

array([[1.42549239e-02, 4.67399561e-06, 1.42897367e-03, 6.47307012e-03,
        1.28015097e-02, 9.17692296e-01, 2.52935849e-02, 9.07885113e-05,
        2.08619649e-02, 1.09821436e-03]])

In [ ]:
#Округляем ответ до 1000, при этом File20 содержит только одну нулевую строку, где первый [0] означает нулевая строка, а второй [0] номер столбца вектора 0 до 9.
#Необходимо найти вероятность отнесения изображения 5 к ее реальному значению (5), это пятый столбец.
print(round(clf_logis.predict_proba(File24)[0][5],3))

0.918


Введите вероятность отнесения изображения file8 к назначенному классу деревом принятия решений.

In [ ]:
#Предсказание для file8 из таблицы
clf_tree.fit(X_train, y_train)
#Bыбор строки из таблицы
File8 = data_test.loc[7,:].to_frame().T
File8 = File8.drop(['Label'], axis=1)
File8 = modelPCA.transform(File8)
#Oтбор данных для предикторов и предсказание
clf_tree.predict_proba(File8)

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [ ]:
#Округляем ответ до 1000, при этом File8 содержит только одну нулевую строку, где первый [0] означает нулевая строка, а второй [0] номер столбца вектора 0 до 9.
#Необходимо найти вероятность отнесения изображения 9 к ее реальному значению (9), это девятый столбец.
print(round(clf_tree.predict_proba(File8)[0][9],3))

1.0
